In [1]:
# boiler plate
from collections import Counter
import pickle
import os
from importlib import reload
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler


import ipdb
import mytf.utils as mu

In [2]:
with open('models/2019-07-21T1815-UTC-outdata-SUBSET50k.pkl', 'rb') as fd:
    julydata = pickle.load(fd)


In [7]:
def full_model(dropout, recurrent_dropout, units):
    with tf.name_scope('themodel'):
        model = tf.keras.Sequential([
        tf.keras.layers.LSTM(units,
                             dropout=dropout, 
                             recurrent_dropout=recurrent_dropout,
                             batch_input_shape=(None, 256, 3), 
                  ),
        # 4 because 'A', 'B', 'C', 'D'.
        tf.keras.layers.Dense(4)])
        return model


In [5]:
class_weights = {0: 1.0, 1: 1.3, 2: 1.0, 3: 1.0}
training_indices = np.arange(0, julydata['x_train'].shape[0], 1)
print('training indices...', training_indices.shape)

train_vec, label_vec, weights_vec = mu.helper_build_dataset_weighty_v3(
    julydata,
    training_indices, 
    class_weights,
    batch_size=1000)

training indices... (50000,)
Start build v3: .. doesnt add up to 1.0
num slices 50
size_remainder,  0
Counter({0: 540, 2: 386, 3: 52, 1: 22})
weights_per_class,  Tensor("truediv:0", shape=(4,), dtype=float32)
1006.5999999999997
Counter({0: 490, 2: 440, 3: 53, 1: 17})
weights_per_class,  Tensor("truediv_1:0", shape=(4,), dtype=float32)
1005.0999999999997
Counter({0: 517, 2: 412, 3: 50, 1: 21})
weights_per_class,  Tensor("truediv_2:0", shape=(4,), dtype=float32)
1006.2999999999997
Counter({0: 526, 2: 398, 3: 56, 1: 20})
weights_per_class,  Tensor("truediv_3:0", shape=(4,), dtype=float32)
1005.9999999999997
Counter({0: 499, 2: 438, 3: 46, 1: 17})
weights_per_class,  Tensor("truediv_4:0", shape=(4,), dtype=float32)
1005.0999999999997
Counter({0: 521, 2: 411, 3: 53, 1: 15})
weights_per_class,  Tensor("truediv_5:0", shape=(4,), dtype=float32)
1004.4999999999997
Counter({0: 537, 2: 392, 3: 59, 1: 12})
weights_per_class,  Tensor("truediv_6:0", shape=(4,), dtype=float32)
1003.5999999999997
Coun

In [6]:
import datetime; import pytz
def quickts():
    return datetime.datetime.utcnow().replace(tzinfo=pytz.UTC).strftime('%Y-%m-%dT%H%M%S')
quickts()

'2019-08-11T202305'

In [11]:
"lr_%.0E,%s,%s" % (.00000001234, 4, 5)

'lr_1E-08,4,5'

In [15]:
LOGDIR = './runtime/'

def aug11_model_two(subdir, hparams):
    learning_rate = hparams['learning_rate']
    lstm_units = hparams['lstm_units']
    lstm_dropout = hparams['lstm_dropout']
    lstm_recurrent_dropout = hparams['lstm_recurrent_dropout']

    hparams_string = "lr_%.0E,units=%s,dro=%s" % (learning_rate,
                                       lstm_units,
                                       lstm_dropout,
                                       )

    tf.reset_default_graph()
    sess = tf.Session()
    run_metadata = tf.RunMetadata()
        
    # Setup placeholders, and reshape the data
    x = tf.placeholder(tf.float32, shape=[None, 256, 3], name="x")
        
    # x_image = tf.reshape(x, [-1, 28, 28, 1])
    # ? tf.summary.image('input', x_image, 3)
    y = tf.placeholder(tf.int64, shape=[None, 1], name="labels")

    themodel = full_model(dropout=lstm_dropout,
                          recurrent_dropout=lstm_recurrent_dropout,
                          units=lstm_units)
    logits = themodel(x) # training=True ?
    
    tf.summary.histogram("logits_0", logits[:, 0]) # NOTE might need to use mean?
    tf.summary.histogram("logits_1", logits[:, 1])
    tf.summary.histogram("logits_2", logits[:, 2])
    tf.summary.histogram("logits_3", logits[:, 3])


    with tf.name_scope("xent"):
        xent = tf.losses.sparse_softmax_cross_entropy(y, logits, 
                                                      #weights=weights
                                                     )
        # xent = tf.reduce_mean(
        #         tf.nn.softmax_cross_entropy_with_logits(
        #                 logits=logits, labels=y), name="xent")
        tf.summary.scalar("xent", xent)
        tf.summary.histogram("x_entropy", xent)
        

    with tf.name_scope("train"):
        train_step = tf.train.AdamOptimizer(learning_rate).minimize(xent)

    with tf.name_scope("accuracy"):
        correct_prediction = tf.equal(tf.argmax(logits, 1), y)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar("accuracy", accuracy)

    summ = tf.summary.merge_all()


    # embedding = tf.Variable(tf.zeros([1024, embedding_size]), name="test_embedding")
    # assignment = embedding.assign(embedding_input)
    saver = tf.train.Saver()

    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(LOGDIR 
                                   + '/' 
                                   + subdir 
                                   + '/'
                                  + hparams_string) # <--hyperparam too.
    writer.add_graph(sess.graph)
    
    # writer.add_run_metadata(run_metadata, 'blahhmmm')
    

    for i in range(50): # 45 or 50?
        left, right = train_vec[i], np.resize(label_vec[i], (1000,1))  # , weights_vec


        if i % 5 == 0:
            [train_accuracy, s] = sess.run([accuracy, summ], 
                                           feed_dict={x: left,
                                                      y: right})
            writer.add_summary(s, i)

        if i % 5 == 0:
            saver.save(sess, os.path.join(LOGDIR, subdir, "model.ckpt"), i) # nohparams?
        sess.run(train_step, feed_dict={x: left, y: right}, )

        
def main(subdir):
    lstm_recurrent_dropout = 0.2
    for learning_rate in [1e-3, 1e-4]:
        for dropout in [0.2, 0.99]:
            for lstm_units in [8, 16, 64]:

                hparams = {'lstm_recurrent_dropout': lstm_recurrent_dropout,
                         'learning_rate': learning_rate,
                         'lstm_units': lstm_units,
                         'lstm_dropout': dropout}

                aug11_model_two(subdir, hparams)
    print('Done training!')
    print('Run `tensorboard --logdir=%s` to see the results.' % LOGDIR + '/' + subdir)
    print('Running on mac? If you want to get rid of the dialogue asking to give '
                'network permissions to TensorBoard, you can provide this flag: '
                '--host=localhost')




In [16]:
%%time
main(quickts())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use standard file APIs to delete files with this prefix.
Done training!
Run `tensorboard --logdir=./runtime/` to see the results./2019-08-11T203728
Running on mac? If you want to get rid of the dialogue asking to give network permissions to TensorBoard, you can provide this flag: --host=localhost
CPU times: user 30min 2s, sys: 11min 47s, total: 41min 50s
Wall time: 10min 31s


#### Looking at tensorboard
* It's not easy to discern any good pattern, but it appears the firing in the histogram for `logits=1` is sharper for `lr_1E-03,units=64,dro=0.2` . 
<img src="notes/assets/Screen%20Shot%202019-08-11%20at%208.13.54%20PM.png" width="85%" height="40%">

* It's actually also kind of sharp for `lr_1E-03,units=8,dro=0.99` , below, but it's just sharp around `0` which means basically nothing interesting has happened here.

<img src="notes/assets/Screen%20Shot%202019-08-11%20at%208.33.05%20PM.png" width="85%" height="40%">


* But although `lr_1E-03,units=64,dro=0.2` is sharp, it is still negative, so it will always lose in the argmax.
* I wonder what happens if I were to take the absolute value. I don't actually know what data the firing is happening for.




In [ ]:

def hmm(logits, labels):
    indices_per_class = {
        c: [i for (i, x) in enumerate(labels)
           if x == c]
        for c in [0, 1, 2, 3]
    }
    
    

